In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# data

In [ ]:
# 归一化
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

def split_data(X, y, X_mask, y_mask, city_list, item_list):
    # 2020 前100个城市
    id = np.where(X[::26, 0, 5] == 1)[0][:300]*26
    # 存储所有 group 的坐标
    ids = []
    for start in id:
        group_ids = np.arange(start, start + 26)
        ids.append(group_ids)
    # 将所有坐标合并成一个大数组
    train_indices = np.concatenate(ids)

    # 2021-2022 和2020其他城市
    val_indices = np.setdiff1d(np.arange(X.shape[0]), train_indices)
    test_indices = val_indices
    
    X_train, X_val, X_test = X[train_indices], X[val_indices], X[test_indices]
    y_train, y_val, y_test = y[train_indices], y[val_indices], y[test_indices]
    X_mask_train, X_mask_val, X_mask_test = X_mask[train_indices], X_mask[val_indices], X_mask[test_indices]
    y_mask_train, y_mask_val, y_mask_test = y_mask[train_indices], y_mask[val_indices], y_mask[test_indices]
    city_train, city_val, city_test = city_list[train_indices], city_list[val_indices], city_list[test_indices]
    item_train, item_val, item_test = item_list[train_indices], item_list[val_indices], item_list[test_indices]
    
    return (X_train, y_train, X_mask_train, y_mask_train), (X_val, y_val, X_mask_val, y_mask_val), (X_test, y_test, X_mask_test, y_mask_test), (city_train, city_val, city_test), (item_train, item_val, item_test)

def normalize_data(X_train, X_val, X_test, y_train, y_val, y_test, columns_to_normalize):
    scalers = {}

    # Normalize specific columns of X
    for col in columns_to_normalize:
        scaler = StandardScaler()
        X_train[:, :, col] = scaler.fit_transform(X_train[:, :, col])
        X_val[:, :, col] = scaler.transform(X_val[:, :, col])
        X_test[:, :, col] = scaler.transform(X_test[:, :, col])
        scalers[col] = scaler
    
    # Normalize the first column of X and y together
    first_col_scaler = StandardScaler()
    combined_train = np.hstack((X_train[:, :, 0], y_train))
    combined_val = np.hstack((X_val[:, :, 0], y_val))
    combined_test = np.hstack((X_test[:, :, 0], y_test))
    
    combined_train = first_col_scaler.fit_transform(combined_train)
    combined_val = first_col_scaler.transform(combined_val)
    combined_test = first_col_scaler.transform(combined_test)
    
    X_train[:, :, 0] = combined_train[:, :X_train.shape[1]]
    y_train = combined_train[:, X_train.shape[1]:]
    X_val[:, :, 0] = combined_val[:, :X_val.shape[1]]
    y_val = combined_val[:, X_val.shape[1]:]
    X_test[:, :, 0] = combined_test[:, :X_test.shape[1]]
    y_test = combined_test[:, X_test.shape[1]:]
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), scalers

In [ ]:
data = np.load('./data/unsplit_data.npz', allow_pickle=True)#_deli
X = data['X']
y = data['y']
X_mask = data['X_mask']
y_mask = data['y_mask']
city_list = data['city_list']
item_list = data['item_list']
item_to_idx = data['item_to_idx'].item()
idx_to_item = data['idx_to_item'].item()

In [ ]:
# Split data
(X_train, y_train, X_mask_train, y_mask_train), (X_val, y_val, X_mask_val, y_mask_val), (X_test, y_test, X_mask_test, y_mask_test), (city_train, city_val, city_test), (item_train, item_val, item_test) = split_data(X, y, X_mask, y_mask, city_list, item_list)

# Normalize data
# 2020-2022
columns_to_normalize = [1, 2, 3, 4]
# # 2023
# columns_to_normalize = []
(X_train, y_train), (X_val, y_val), (X_test, y_test), scalers = normalize_data(X_train, X_val, X_test, y_train, y_val, y_test, columns_to_normalize)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape, city_train.shape, city_val.shape, city_test.shape, item_train.shape

In [ ]:
# 检查GPU可用性
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

mode = 'muiti_dim'
if mode == 'muiti_dim':
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.float32).to(device)#.squeeze(-1)
    X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.float32).to(device)
    X_mask_train = torch.tensor(X_mask_train, dtype=torch.float32).to(device)
    y_mask_train = torch.tensor(y_mask_train, dtype=torch.float32).to(device)
    X_mask_val = torch.tensor(X_mask_val, dtype=torch.float32).to(device)
    y_mask_val = torch.tensor(y_mask_val, dtype=torch.float32).to(device)
    X_mask_test = torch.tensor(X_mask_test, dtype=torch.float32).to(device)
    y_mask_test = torch.tensor(y_mask_test, dtype=torch.float32).to(device)
    # city_train = torch.tensor(city_train, dtype=torch.float32).to(device)
    # city_val = torch.tensor(city_val, dtype=torch.float32).to(device)
    # city_test = torch.tensor(city_test, dtype=torch.float32).to(device)
    item_train = torch.tensor(item_train, dtype=torch.long).to(device)
    item_val = torch.tensor(item_val, dtype=torch.long).to(device)
    item_test = torch.tensor(item_test, dtype=torch.long).to(device)
elif mode == '1-dim':
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)[:,:,0].unsqueeze(-1)
    y_train = torch.tensor(y_train, dtype=torch.float32).to(device)#.squeeze(-1)
    X_val = torch.tensor(X_val, dtype=torch.float32).to(device)[:,:,0].unsqueeze(-1)
    y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)[:,:,0].unsqueeze(-1)
    y_test = torch.tensor(y_test, dtype=torch.float32).to(device)
    X_mask_train = torch.tensor(X_mask_train, dtype=torch.float32).to(device)[:,:,0].unsqueeze(-1)
    y_mask_train = torch.tensor(y_mask_train, dtype=torch.float32).to(device)
    X_mask_val = torch.tensor(X_mask_val, dtype=torch.float32).to(device)[:,:,0].unsqueeze(-1)
    y_mask_val = torch.tensor(y_mask_val, dtype=torch.float32).to(device)
    X_mask_test = torch.tensor(X_mask_test, dtype=torch.float32).to(device)[:,:,0].unsqueeze(-1)
    y_mask_test = torch.tensor(y_mask_test, dtype=torch.float32).to(device)
    # city_train = torch.tensor(city_train, dtype=torch.float32).to(device)
    # city_val = torch.tensor(city_val, dtype=torch.float32).to(device)
    # city_test = torch.tensor(city_test, dtype=torch.float32).to(device)
    item_train = torch.tensor(item_train, dtype=torch.long).to(device)
    item_val = torch.tensor(item_val, dtype=torch.long).to(device)
    item_test = torch.tensor(item_test, dtype=torch.long).to(device)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape, item_train.shape, X_mask_train.shape)
# 城市
city_to_idx = {city: idx for idx, city in enumerate(set().union(city_train, city_val, city_test))}
idx_to_city = {idx: city for city, idx in city_to_idx.items()}
city_train_indices = [city_to_idx[city] for city in city_train]
city_train = torch.tensor(city_train_indices).to(device)
city_val_indices = [city_to_idx[city] for city in city_val]
city_val = torch.tensor(city_val_indices).to(device)
city_test_indices = [city_to_idx[city] for city in city_test]
city_test = torch.tensor(city_test_indices).to(device)

In [ ]:
train_dataset = TensorDataset(X_train, y_train, X_mask_train, y_mask_train, city_train, item_train)
val_dataset = TensorDataset(X_val, y_val, X_mask_val, y_mask_val, city_val, item_val)
test_dataset = TensorDataset(X_test, y_test, X_mask_test, y_mask_test, city_test, item_test)

# val_dataset1 = TensorDataset(X_val1, y_val1, X_mask_val1, y_mask_val1, city_val1, item_val1)
# val_dataset2 = TensorDataset(X_val2, y_val2, X_mask_val2, y_mask_val2, city_val2, item_val2)

import random
# 设置随机种子
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
batch_size = 16

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# val_loader1 = DataLoader(val_dataset1, batch_size=batch_size)
# val_loader2 = DataLoader(val_dataset2, batch_size=batch_size)

# model

In [ ]:
# LSTM
input_size = 45-8
hidden_size = 64
output_size = 14
epochs = 300

param_path = f'./param/pretrain_bs{batch_size}_hs{hidden_size}'

# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.impact = nn.Linear(8, hidden_size//8)
        self.fc = nn.Linear(hidden_size+hidden_size//8, output_size)
    
    def forward(self, x, x_mask):
        cols1 = [4] + list(range(8, 15))
        cols2 = [i for i in range(45) if i not in cols1]
        # 应用输入掩码
        x = x * x_mask
        x1 = x[:, 0, cols1]
        x2 = x[:, :, cols2]
        _, (hn, _) = self.lstm(x2) # b*d
        x1 = self.impact(x1) # b*d'
        out = self.fc(torch.cat((hn[-1], x1), dim=1))
        return out, hn[-1], x1

In [ ]:
# TCN
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()
        
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class TCNModel(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size=2, dropout=0.2):
        super(TCNModel, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout)
        # self.input = nn.Linear(1, 1)
        self.impact = nn.Linear(8, num_channels[-1]//4)
        self.fc = nn.Linear(num_channels[-1]+num_channels[-1]//4, output_size)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, x, x_mask):
        # x:b*n*d
        cols1 = [4] + list(range(8, 15))
        # cols2 = [i for i in range(45) if i not in cols1]
        cols2 = 0 # [0] + list(range(5, 8)) + list(range(-4, 0)) # list(range(0, 45))
        # 应用输入掩码
        x = x * x_mask
        x = x.permute(0, 2, 1)
        x1 = x[:, cols1, 0]
        x2 = x[:, cols2, :].unsqueeze(1) # b*d*n
        # x2 = self.input(x2.permute(0,2,1)).permute(0,2,1)
        
        y = self.tcn(x2)[:, :, -1] # b*d(*n)
        x1 = self.impact(x1) # b*d'
        out = self.fc(torch.cat((y, x1), dim=1))
        # out = self.linear(y)

        return out, y, x1

In [ ]:
input_size = 1#45-8
output_size = 14
num_channels = [32, 64, 32]# [64, 64, 64]
kernel_size = 2
dropout = 0.2
epochs = 20
param_path = f'./param/pretrain_bs{batch_size}_{num_channels}'
deli_param_path = f'./param/pretrain_deli_bs{batch_size}_{num_channels}'
# param_path = f'./param/pretrain_deli_bs{batch_size}_{num_channels}'
# deli_param_path = f'./param/pretrain_bs{batch_size}_{num_channels}'

# 实例化模型
pretrained_model = TCNModel(input_size, output_size, num_channels, kernel_size, dropout).to(device)
deli_pretrained_model = TCNModel(input_size, output_size, num_channels, kernel_size, dropout).to(device)

if not os.path.exists(param_path):
    os.makedirs(param_path)
if not os.path.exists(deli_param_path):
    os.makedirs(deli_param_path)

# 定义损失函数和优化器
criterion = nn.MSELoss(reduction='none')
optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=0.001)

In [ ]:
# 训练模型
best_val_loss = float('inf')
for epoch in range(epochs):
    pretrained_model.train()
    train_loss = 0
    for batch_X, batch_y, batch_X_mask, batch_y_mask, _, _ in train_loader:
        optimizer.zero_grad()
        outputs, _, _ = pretrained_model(batch_X, batch_X_mask)
        loss = criterion(outputs, batch_y)
        loss = (loss * batch_y_mask).mean()  # 应用输出掩码
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    if (epoch + 1) % 1 == 0:
        pretrained_model.eval()
        val_loss = 0
        mae, mape = 0, 0
        with torch.no_grad():
            for val_X, val_y, val_X_mask, val_y_mask, _, _ in val_loader:
                val_outputs, _, _ = pretrained_model(val_X, val_X_mask)
                val_loss_batch = criterion(val_outputs, val_y)
                mae_batch = torch.mean(torch.abs(val_outputs - val_y))# * val_y_mask
                # mape_batch = torch.abs((val_y - val_outputs) / (val_y + 1e-10))# * val_y_mask
                mape_batch = torch.abs((val_y - val_outputs) / val_y)
                mape_batch = torch.where(mape_batch > 5, 0, mape_batch)
                val_loss += (val_loss_batch * val_y_mask).mean().item()
                mae += (mae_batch * val_y_mask).mean().item()
                mape += (mape_batch * val_y_mask).mean().item()
        val_loss /= len(val_loader)
        mae /= len(val_loader)
        mape /= len(val_loader)
        print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, mae: {mae:.4f}, mape: {mape:.4f}')
        # 保存 LSTM 模型参数
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(pretrained_model.state_dict(), deli_param_path+f'/pretrained_model_{epoch+1}.pth')
            print("Model saved!")

# similar model

In [ ]:
param_file = param_path + '/pretrained_model_16.pth' #1 16

pretrain_loader = DataLoader(train_dataset, batch_size=X_train.shape[0])
# 加载 LSTM 模型参数
pretrained_model.load_state_dict(torch.load(param_file))
pretrained_model.eval()

for param in pretrained_model.parameters():
    param.requires_grad = False

for batch_X, batch_y, batch_X_mask, batch_y_mask, _, _ in pretrain_loader:
    with torch.no_grad():
        _, pretrained_embeddings, pretrained_impact = pretrained_model(batch_X, batch_X_mask)
print(pretrained_embeddings.shape, pretrained_impact.shape)

In [ ]:
pretrained_embeddings = pretrained_embeddings.reshape(-1, 26, 32)#.reshape(26, -1, 32).transpose(0, 1)
pretrained_impact = pretrained_impact.reshape(-1, 26, 8)
print(pretrained_embeddings.shape, pretrained_impact.shape)

### deli

In [ ]:
deli_param_file = deli_param_path + '/pretrained_model_2.pth' # 2

deli_pretrain_loader = DataLoader(train_dataset, batch_size=X_train.shape[0])
# 加载 LSTM 模型参数
deli_pretrained_model.load_state_dict(torch.load(deli_param_file))
deli_pretrained_model.eval()

for param in pretrained_model.parameters():
    param.requires_grad = False

for batch_X, batch_y, batch_X_mask, batch_y_mask, _, _ in deli_pretrain_loader:
    with torch.no_grad():
        _, deli_pretrained_embeddings, deli_pretrained_impact = deli_pretrained_model(batch_X, batch_X_mask)
print(deli_pretrained_embeddings.shape, deli_pretrained_impact.shape)

In [ ]:
deli_pretrained_embeddings = deli_pretrained_embeddings.reshape(-1, 26, 32)#.reshape(26, -1, 32).transpose(0, 1)
deli_pretrained_impact = deli_pretrained_impact.reshape(-1, 26, 8)
print(deli_pretrained_embeddings.shape, deli_pretrained_impact.shape)

### LLM+similar

In [ ]:
llm_embed_tensor = torch.load('./data/embed_tensor.pt').to(device)
print(llm_embed_tensor.shape)

In [ ]:
def select_top_k_similar(target_vectors, reference_vectors, top_k=20):
    # target_vectors: b*d
    # reference_vectors: b*200*d
    b, d = target_vectors.shape
    
    # # 归一化向量
    # target_vectors = target_vectors / target_vectors.norm(dim=1, keepdim=True)
    # reference_vectors = reference_vectors / reference_vectors.norm(dim=2, keepdim=True)
    
    # 计算点积
    dot_product = torch.bmm(reference_vectors, target_vectors.unsqueeze(2)).squeeze(2)  # b*200

    # 计算 A 和 B 的范数
    norm_A = torch.norm(target_vectors, dim=1, keepdim=True)  # [b, 1]
    norm_B = torch.norm(reference_vectors, dim=2)  # [b, 200]
    
    # 计算余弦相似度
    cosine_similarity = dot_product / (norm_A * norm_B)  # [b, 200]
    
    # 获取相似度最高的 top_k 个索引
    top_k_values, top_k_indices = torch.topk(cosine_similarity, top_k, dim=1)
    
    return top_k_indices # b*top_k

# # 示例用法
# b, d = 5, 10  # 例如，5 个目标向量，每个向量 10 维
# target_vectors = torch.rand(b, d)
# reference_vectors = torch.rand(b, 200, d)

# top_k_indices = select_top_k_similar(target_vectors, reference_vectors)
# print(top_k_indices)

In [ ]:
def extract_top_k_vectors(reference_vectors, top_k_indices):
    # reference_vectors: b*200*d
    # top_k_indices: b*20
    b, num_refs, d = reference_vectors.shape
    _, top_k = top_k_indices.shape
    
    # 使用高级索引提取所需的向量
    # 通过扩展批次维度来匹配索引
    batch_indices = torch.arange(b).unsqueeze(1).expand(-1, top_k)
    
    # 提取对应的向量
    top_k_vectors = reference_vectors[batch_indices, top_k_indices]
    
    return top_k_vectors # b*top_k*d

# # 示例用法
# b, num_refs, d = 5, 200, 10
# reference_vectors = torch.rand(b, num_refs, d)
# top_k_indices = torch.randint(0, num_refs, (b, 20))

# top_k_vectors = extract_top_k_vectors(reference_vectors, top_k_indices)
# print(top_k_vectors.shape)  # 应输出: torch.Size([5, 20, 10])

In [ ]:
# LLM模型
import torch.nn.functional as F
import time
import logging
from datetime import datetime

alpha = 0.1
epochs = 20
hidden_size = num_channels[-1]
all_city = city_train[::26]

# if not os.path.exists('./logs'):
#     os.makedirs('./logs')
# # 获取根记录器
# logger = logging.getLogger()
# # 移除已有的处理器
# if logger.hasHandlers():
#     logger.handlers.clear()
# # 配置日志记录器
# current_time = datetime.now().strftime("%Y-%m-%d_%H-%M")
# logging.basicConfig(
#     filename=f'./logs/CLlog_bs{batch_size}_epo{epochs}_{current_time}.log',#
#     level=logging.INFO,
#     format='%(asctime)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s'
# )

def contrastive_loss(embedding1, embedding2, temperature=0.5):
    # Normalize the embeddings
    embedding1 = F.normalize(embedding1, dim=1)
    embedding2 = F.normalize(embedding2, dim=1)
    
    # Compute similarity scores
    similarity_matrix = torch.mm(embedding1, embedding2.T) / temperature
    
    # Create labels (positive samples on diagonal)
    labels = torch.arange(similarity_matrix.size(0)).to(similarity_matrix.device)
    
    # Compute the loss
    loss = F.cross_entropy(similarity_matrix, labels)
    return loss

class SimModel(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(SimModel, self).__init__()
        self.x_mlp = nn.Linear(hidden_size, hidden_size)
        self.all_xs_mlp = nn.Linear(hidden_size, hidden_size)
        self.impact_mlp = nn.Linear(26, 1)
        self.post_mlp = nn.Linear(hidden_size//4, hidden_size//4)
        self.fc = nn.Linear(hidden_size*2, output_size)
        self.relu = nn.ReLU()

        self.sim_mlp = nn.Linear(hidden_size+hidden_size//4, hidden_size)#+hidden_size
        # self.llm_mlp = nn.Linear(768, hidden_size//4)
        self.llm_mlp = nn.Sequential(
            nn.Linear(768, hidden_size//2),  # 第一层全连接层
            nn.ReLU(),                           # ReLU 激活函数
            nn.Linear(hidden_size//2, hidden_size)  # 第二层全连接层
        )
        self.deli_mlp = nn.Linear(hidden_size, hidden_size)

    def forward(self, deli, x, impact, city, item, all_xs, all_impact, all_city, epoch):
        # b*d, b*d, b*d', b, b, 300*26*d, 300*26*d', 300, 1
        # pre_attention
        x = self.x_mlp(x)
        all_xs = self.all_xs_mlp(all_xs)
        
        scores1 = torch.einsum('bd,mcd->bmc', x, all_xs) / (hidden_size ** 0.5) # b*300*26
        weights1 = torch.softmax(scores1, dim=-1)
        all_embed = torch.einsum('bmc,mdc->bmd', weights1, all_xs.transpose(1,2)) # b*300*d
        # similar
        top_k_indices = select_top_k_similar(x, all_embed, top_k=10) # b*k
        # reference_city = all_city[top_k_indices] # b*k
        # similar_embed = extract_top_k_vectors(all_embed, top_k_indices) # b*k*d
        similar_impact = all_impact[top_k_indices] # b*k*26*d'  .unsqueeze(-1).unsqueeze(-1).expand(-1, -1, all_impact.shape[-2], all_impact.shape[-1])
        similar_impact = self.impact_mlp(similar_impact.permute(0,1,3,2)).squeeze(-1) # b*k*d'
        
        # post_attention
        similar_impact = self.post_mlp(similar_impact)
        scores2 = torch.einsum('bd,bkd->bk', impact, similar_impact) / (hidden_size ** 0.5)
        weights2 = torch.softmax(scores2, dim=-1)
        sim_embed = torch.einsum('bk,bkd->bd', weights2, similar_impact) # b*d'

        # deli = self.deli_mlp(deli)
        sim_embed = self.sim_mlp(torch.cat((x, sim_embed), dim=1)) # b*d
        # llm_embed
        llm_embed = self.llm_mlp(llm_embed_tensor[city, item])
        
        output = self.fc(torch.cat((sim_embed, llm_embed), dim=1))
        # output = self.fc(self.relu(sim_embed))
        # output = self.fc(self.relu(torch.cat((sim_embed, deli), dim=1)))
        
        return sim_embed, llm_embed, output
        # return sim_embed, None, output

for alpha in [0.1, 0.2, 0.5, 1]: #[0.1]: #
    for temperature in [0.2, 0.5, 1]:# [1]:#
        print(alpha, temperature, '-'*50)

        # 获取根记录器
        logger = logging.getLogger()
        # 移除已有的处理器
        if logger.hasHandlers():
            logger.handlers.clear()
        logging.basicConfig(
            filename=f'./logs/CLlog_alpha{alpha}_tem{temperature}_bs{batch_size}.log',#
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s'
        )

        # 加载 LSTM 模型参数
        pretrained_model.load_state_dict(torch.load(param_file))
        pretrained_model.eval()
        deli_pretrained_model.load_state_dict(torch.load(deli_param_file))
        deli_pretrained_model.eval()
        
        for param in pretrained_model.parameters():
            param.requires_grad = False
        for param in deli_pretrained_model.parameters():
            param.requires_grad = False
        
        sim_model = SimModel(hidden_size, output_size).to(device)
        optimizer = torch.optim.Adam(sim_model.parameters(), lr=0.001)
        
        best_val_loss = float('inf')
        
        for epoch in range(epochs):
            sim_model.train()
            steps = 0
            train_loss = 0
            t0 = time.time()
            for batch_X, batch_y, batch_X_mask, batch_y_mask, city, item in train_loader:
                with torch.no_grad():
                    # B*d
                    _, embeddings, impact = pretrained_model(batch_X, batch_X_mask)
                    _, deli_embeddings, deli_impact = deli_pretrained_model(batch_X, batch_X_mask)
                
                optimizer.zero_grad()
                sim_embed, llm_embed, outputs = sim_model(deli_embeddings, embeddings, impact, city, item, pretrained_embeddings, pretrained_impact, all_city, epoch+1)
                loss = criterion(outputs, batch_y)
                loss = (loss * batch_y_mask).mean() + contrastive_loss(sim_embed, llm_embed, temperature) * alpha
        
                loss.backward()
                optimizer.step()
                steps += 1
                # print(f'epoch: [{epoch+1}], step: [{steps}], Loss: {loss.item():.4f}')
                train_loss += loss.item()
            train_loss /= len(train_loader)
            t1 = time.time()
            # print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, time: {t1-t0}')
            # logging.info(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, time: {t1-t0}')
                
            if (epoch + 1) % 1 == 0:
                # t0 = time.time()
                sim_model.eval()
                val_loss = 0
                mae, mape = 0, 0
                with torch.no_grad():
                    for val_X, val_y, val_X_mask, val_y_mask, val_city, val_item in val_loader:
                        _, embeddings, impact = pretrained_model(val_X, val_X_mask)
                        _, deli_embeddings, deli_impact = deli_pretrained_model(val_X, val_X_mask)
                        _, _, val_outputs = sim_model(deli_embeddings, embeddings, impact, val_city, val_item, pretrained_embeddings, pretrained_impact, all_city, epoch+1)
                        val_loss_batch = criterion(val_outputs, val_y)
                        steps += 1
                        mae_batch = torch.mean(torch.abs(val_outputs - val_y))# * val_y_mask
                        # mape_batch = torch.abs((val_y - val_outputs) / (val_y + 1e-10))# * val_y_mask
                        mape_batch = torch.abs((val_y - val_outputs) / val_y)
                        mape_batch = torch.where(mape_batch > 5, 0, mape_batch)
                        val_loss += (val_loss_batch * val_y_mask).mean().item()
                        mae += (mae_batch * val_y_mask).mean().item()
                        mape += (mape_batch * val_y_mask).mean().item()
                val_loss /= len(val_loader)
                mae /= len(val_loader)
                mape /= len(val_loader)
                print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, mae: {mae:.4f}, mape: {mape:.4f}')
                logging.info(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, mae: {mae:.4f}, mape: {mape:.4f}')
                # 保存 LSTM 模型参数
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    if not os.path.exists(f'./param/CLmodel_bs{batch_size}_{epochs}'):
                        os.makedirs(f'./param/CLmodel_bs{batch_size}_{epochs}')
                    torch.save(sim_model.state_dict(), f'./param/CLmodel_bs{batch_size}_{epochs}/CLmodel_alpha{alpha}_tem{temperature}.pth')
                    # print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, mae: {mae:.4f}, mape: {mape:.4f}')
                    print("Model saved!")

# LLM

In [ ]:
import json
import requests
import ujson as json
import openai
import os

# 填写 申请的 apikey 和 erp
my_api_key = ""
my_erp = ""

os.environ["OPENAI_API_KEY"] = my_api_key

In [ ]:
os.environ["OPENAI_API_BASE"] = ""

In [ ]:
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import logging
import pickle
import asyncio
import nest_asyncio
import re

## prompt找相似城市

In [ ]:
if not os.path.exists('./llmlogs'):
    os.makedirs('./llmlogs')
# 获取根记录器
logger = logging.getLogger()
# 移除已有的处理器
if logger.hasHandlers():
    logger.handlers.clear()
# 配置日志记录器
# current_time = datetime.now().strftime("%Y-%m-%d_%H-%M")
logging.basicConfig(
    filename=f'./llmlogs/similar_city.log',#_bs{batch_size}_epo{epochs}_{current_time}
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

chat_model = ChatOpenAI(model="gpt-4o-mini")

similarcity_dict = {} # {cityidx: [idxs]}
for cityidx in city_to_idx.values():
    
    memory = ConversationBufferMemory()
    conversation = ConversationChain(llm=chat_model, memory=memory)
    # 定义目标城市和参考城市列表
    target_city = f"<city>({idx_to_city[cityidx]})"
    reference_cities = [idx_to_city[idx.item()] for idx in torch.unique(city_train)]
    # 将参考城市列表转换为字符串格式
    reference_cities_str = ", ".join([f"<city>({city})" for city in reference_cities])
    
    prompt = f"有些城市发生了疫情，希望了解这些城市可能的线上单量的变化趋势。我们会给出一个目标城市和一些参考城市，希望你能找到线上单量变化趋势和目标城市最相似的10个参考城市。\n目标城市的名称如下：{target_city}。参考城市的名单如下：{reference_cities_str}。\n请从气候信息，地理信息，人口密度，基础设施，经济活动等方面来分析这些城市在发生疫情后，线上订单未来两周可能的变化趋势。请从参考城市中挑选出10个和目标城市最相似的城市，按相似程度的顺序从高到低排列，格式为<city>(城市名)。注意，挑选的城市必须是在参考城市中出现过的。"
    # prompt = f"目标城市: {target_city}\n参考城市列表: {reference_cities_str}\n请根据城市的地理位置、文化、经济、人口等特征进行分析，列出与目标城市最相似的10个城市，仅限于参考城市列表内："
    response = conversation.run(input=prompt)
    # print("AI:", response)
    
    def validate_and_feedback(response, reference_cities, val_times=0):
        # result_cities = [city.strip() for city in response.split(",")]
        # 正则表达式模式
        pattern = r"<city>\((.*?)\)"
        # 查找所有匹配
        result_cities = re.findall(pattern, response)
        
        validated_cities = [city for city in result_cities if city in reference_cities]
        missing_cities = [city for city in result_cities if city not in reference_cities]
        
        if len(validated_cities) < 10 and val_times < 3:
            val_times += 1
            if len(missing_cities) > 0:
                feedback_prompt = f"你提供的部分城市不在参考城市列表中。缺失的城市: {', '.join(missing_cities)}。请重新生成与目标城市最相似的城市，仅限于参考城市列表内。"
            else:
                feedback_prompt = f"你提供的城市格式有问题，请注意格式必须为<city>(城市名)。请重新生成与目标城市最相似的城市。"
            response = conversation.run(input=feedback_prompt)
            # print("AI:", response)
            return validate_and_feedback(response, reference_cities, val_times)
        return validated_cities
    
    validated_cities = validate_and_feedback(response, reference_cities)
    if len(validated_cities) < 10:
        print(f"与{idx_to_city[cityidx]} 目标城市最相似的{len(validated_cities)}个城市:", validated_cities)
    logging.info(memory.buffer)
    similarcity_dict[cityidx] = [city_to_idx[city] for city in validated_cities]

# 将字典保存为 Pickle 文件
with open('./data/similarcity_dict.pkl', 'wb') as pickle_file:
    pickle.dump(similarcity_dict, pickle_file)


In [ ]:
with open('./data/similarcity_dict.pkl', 'rb') as pickle_file:
    similarcity_dict = pickle.load(pickle_file)

In [ ]:
similarcity_dict

## prompt分析可能趋势

In [ ]:
num_dict = {} # {(city, item): num}
train_dict = {(city_train[i].item(), i%26): torch.cat((X_train[i, :, 0], y_train[i, :])).cpu() for i in range(X_train.shape[0])}
val_dict = {(city_val[i].item(), i%26): torch.cat((X_val[i, :, 0], y_val[i, :])).cpu() for i in range(X_val.shape[0])}
num_dict = {**train_dict, **val_dict}
# 将字典保存为 Pickle 文件
with open('./data/num_dict.pkl', 'wb') as pickle_file:
    pickle.dump(num_dict, pickle_file)
len(num_dict)/26

In [ ]:
with open('./data/num_dict.pkl', 'rb') as pickle_file:
    num_dict = pickle.load(pickle_file)

In [ ]:
# BERT提取文本嵌入
from transformers import BertTokenizer, BertModel
import torch

def text_embed(text):
    # 加载预训练的BERT模型和分词器
    #t1 = time.time()
    tokenizer = BertTokenizer.from_pretrained('./local-bert-base-uncased') #'bert-base-uncased'
    #t2 = time.time()
    model = BertModel.from_pretrained('./local-bert-base-uncased').to(device)
    #t3 = time.time()

    # 将文本编码为输入ID和注意力掩码
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    #t4 = time.time()
    
    # 获取BERT模型的输出
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 获取最后一个隐藏层的输出
    last_hidden_states = outputs.last_hidden_state
    
    # 通常，我们可以使用CLS token的嵌入作为句子的表示
    sentence_embedding = last_hidden_states[:, 0, :].squeeze()
    #t5 = time.time()
    #print(t5-t4, t4-t3, t3-t2, t2-t1)
    return sentence_embedding
    
# # 输入文本
# text = ["This is an example sentence.", "agrret"]
# sentence_embedding = text_embed(text)
# print(sentence_embedding[0].shape) # 768

In [ ]:
def extract_summary(text):
    # 查找 "@总结:" 的起始位置
    start_index = text.find("@总结：")
    
    # 如果找到了 "@总结:"
    if start_index != -1:
        # 加上 "@总结:" 的长度，计算内容的起始位置
        content_start = start_index + len("@总结：")
        # 提取并返回从该位置开始的内容
        return text[content_start:].strip()
    else:
        # 如果没有找到 "@总结:"，返回默认值
        return text

In [ ]:
def normalize_tensor(tensor):
    # 使用 torch 的功能计算最小值和最大值
    min_val = torch.min(tensor)
    max_val = torch.max(tensor)
    
    # 避免除以零
    if max_val == min_val:
        return torch.zeros_like(tensor)
    
    # 使用广播机制进行归一化
    normalized_tensor = (tensor - min_val) / (max_val - min_val)
    return normalized_tensor

# 对字典中的每个张量进行归一化
num_dict = {key: normalize_tensor(tensor) for key, tensor in num_dict.items()}

### 同步调用

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
import time
from transformers import logging as transformers_logging
import warnings

# 忽略所有警告
warnings.filterwarnings("ignore")
# 设置transformers的日志级别为ERROR
transformers_logging.set_verbosity_error()

if not os.path.exists('./llmlogs'):
    os.makedirs('./llmlogs')
# 获取根记录器
logger = logging.getLogger()
# 移除已有的处理器
if logger.hasHandlers():
    logger.handlers.clear()
# 配置日志记录器
# current_time = datetime.now().strftime("%Y-%m-%d_%H-%M")
logging.basicConfig(
    filename=f'./llmlogs/response_dict.log',#_bs{batch_size}_epo{epochs}_{current_time}
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

chat_model = ChatOpenAI(model="gpt-4o-mini")
embed_dict = {}
response_dict = {}

for cityidx in city_to_idx.values(): # [1]:
    t1 = time.time()
    for item in range(26): # [0]:
        cityidxlist = similarcity_dict[cityidx]
        citynumlist = [f"{idx_to_city[idx]}: 封控前单量：{[round(num, 4) for num in num_dict[(idx, item)][:14].numpy().tolist()]}，封控后单量：{[round(num, 4) for num in num_dict[(idx, item)][14:].numpy().tolist()]}" for idx in cityidxlist]
        citynumstr = "。".join(citynumlist)

        memory = ConversationBufferMemory()
        conversation = ConversationChain(llm=chat_model, memory=memory)

        prompt = f"有些城市发生了疫情，城市封控前后14天的{idx_to_item[item]}类商品的销量（经过了归一化）如下。{citynumstr}。已知{idx_to_city[cityidx]}和上述城市的销量变化相似，请分析{idx_to_city[cityidx]}在封控之后14天内销量的可能变化趋势。给出分析和总结，格式为@分析：（分析内容）@总结：（总结内容）。总结部分仅给出上升下降趋势，幅度，时间，如何波动等销量变化信息，不要提及其他城市，尽量简洁清晰。"
        
        response = conversation.run(input=prompt)
        logging.info(f"{cityidx}, {item} \n{prompt} \n\n{response} \n")
        # sentence_embedding = text_embed(extract_summary(response))
        # embed_dict[(cityidx, item)] = sentence_embedding
        response_dict[(cityidx, item)] = response
        # print(sentence_embedding)
    t2 = time.time()
    print(cityidx, idx_to_city[cityidx], t2-t1)


with open('./data/response_dict.pkl', 'wb') as pickle_file:
    pickle.dump(response_dict, pickle_file)
    
# 将字典保存为 Pickle 文件
# with open('./data/embed_dict.pkl', 'wb') as pickle_file:
#     pickle.dump(embed_dict, pickle_file)

### 异步调用

In [ ]:
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
import asyncio
import nest_asyncio
import re

# 允许在 Jupyter Notebook 中嵌套事件循环
nest_asyncio.apply()
        
async def async_LLM_similar(cityidx, item_list, target_city):
    prompt_s1_sys = "你是一个经济分析师。"
    prompt_s1_user = """
    有些城市发生了疫情，城市封控前后14天的{item}类商品的销量（经过了归一化）如下。{citynumstr}。已知{target_city}和上述城市的销量变化相似，请分析{target_city}在封控之后14天内销量的可能变化趋势。给出分析和总结，格式为@分析：（分析内容）@总结：（总结内容）。总结部分仅给出上升下降趋势，幅度，时间，如何波动等销量变化信息，不要提及其他城市，尽量简洁清晰。
    """
    
    prompt_s1_sys = ChatMessagePromptTemplate.from_template(role='system', template=prompt_s1_sys)
    prompt_s1_user = ChatMessagePromptTemplate.from_template(role='user', template=prompt_s1_user)
    prompt_s1 = ChatPromptTemplate(messages = [prompt_s1_sys, prompt_s1_user]) # 传入 msg 列表 构成 多段输入
    llm_s1 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5, request_timeout=120)
    chain_s1 = LLMChain(llm=llm_s1, prompt=prompt_s1)
    
    async def async_chain_s1(item, citynumstr, target_city, loop_times=3):
        cur_times = 0
        while(cur_times < loop_times):
            try:
                arr_res_multi = await chain_s1.arun({'item':item, 'citynumstr':citynumstr, 'target_city':target_city})
                return arr_res_multi
            except:
                cur_times += 1
        return '' # 超过最大尝试次数后 返回 空字符串
    
    async def async_chain_s1_main(strings_list, target_city):
        tasks = [async_chain_s1(item, citynumstr, target_city) for item, citynumstr in strings_list]
        responses = await asyncio.gather(*tasks)
        
        for i, response in enumerate(responses):
            prompt = f"有些城市发生了疫情，城市封控前后14天的{idx_to_item[i]}类商品的销量（经过了归一化）如下。{strings_list[i][1]}。已知{target_city}和上述城市的销量变化相似，请分析{target_city}在封控之后14天内销量的可能变化趋势。给出分析和总结，格式为@分析：（分析内容）@总结：（总结内容）。总结部分仅给出上升下降趋势，幅度，时间，如何波动等销量变化信息，不要提及其他城市，尽量简洁清晰。"
            logging.info(f"{cityidx}, {i} \n{prompt} \n\n{response} \n")
            response_dict[(cityidx, i)] = response
            

    citynumstr_list = []
    for item in range(26):
        cityidxlist = similarcity_dict[cityidx]
        citynumlist = [f"{idx_to_city[idx]}: 封控前单量：{[round(num, 4) for num in num_dict[(idx, item)][:14].numpy().tolist()]}，封控后单量：{[round(num, 4) for num in num_dict[(idx, item)][14:].numpy().tolist()]}" for idx in cityidxlist]
        citynumstr = "。".join(citynumlist)
        citynumstr_list.append(citynumstr)
    
    await async_chain_s1_main(list(zip(item_list, citynumstr_list)), target_city)

In [ ]:
if not os.path.exists('./llmlogs'):
    os.makedirs('./llmlogs')
# 获取根记录器
logger = logging.getLogger()
# 移除已有的处理器
if logger.hasHandlers():
    logger.handlers.clear()
# 配置日志记录器
# current_time = datetime.now().strftime("%Y-%m-%d_%H-%M")
logging.basicConfig(
    filename=f'./llmlogs/response_dict_async.log',#_bs{batch_size}_epo{epochs}_{current_time}
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

response_dict = {}
for cityidx in city_to_idx.values(): #[0,1]:# 
    target_city = idx_to_city[cityidx]
    t1 = time.time()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(async_LLM_similar(cityidx, [idx_to_item[idx] for idx in range(26)], target_city))
    t2 = time.time()
    print(cityidx, idx_to_city[cityidx], t2-t1)

with open('./data/response_dict_async.pkl', 'wb') as pickle_file:
    pickle.dump(response_dict, pickle_file)

In [ ]:
with open('./data/response_dict_async.pkl', 'rb') as pickle_file:
    response_dict = pickle.load(pickle_file)

In [ ]:
# 异步调用丢失的值
no_response = []
for key, response in response_dict.items():
    if response == "":
        no_response.append(key)
print(len(no_response))

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
import time
from transformers import logging as transformers_logging
import warnings

# 忽略所有警告
warnings.filterwarnings("ignore")
# 设置transformers的日志级别为ERROR
transformers_logging.set_verbosity_error()

if not os.path.exists('./llmlogs'):
    os.makedirs('./llmlogs')
# 获取根记录器
logger = logging.getLogger()
# 移除已有的处理器
if logger.hasHandlers():
    logger.handlers.clear()
# 配置日志记录器
# current_time = datetime.now().strftime("%Y-%m-%d_%H-%M")
logging.basicConfig(
    filename=f'./llmlogs/no_response_dict.log',#_bs{batch_size}_epo{epochs}_{current_time}
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

chat_model = ChatOpenAI(model="gpt-4o-mini")
no_response_dict = {}

for i, (cityidx, item) in enumerate(no_response):
    t1 = time.time()
    cityidxlist = similarcity_dict[cityidx]
    citynumlist = [f"{idx_to_city[idx]}: 封控前单量：{[round(num, 4) for num in num_dict[(idx, item)][:14].numpy().tolist()]}，封控后单量：{[round(num, 4) for num in num_dict[(idx, item)][14:].numpy().tolist()]}" for idx in cityidxlist]
    citynumstr = "。".join(citynumlist)

    memory = ConversationBufferMemory()
    conversation = ConversationChain(llm=chat_model, memory=memory)

    prompt = f"有些城市发生了疫情，城市封控前后14天的{idx_to_item[item]}类商品的销量（经过了归一化）如下。{citynumstr}。已知{idx_to_city[cityidx]}和上述城市的销量变化相似，请分析{idx_to_city[cityidx]}在封控之后14天内销量的可能变化趋势。给出分析和总结，格式为@分析：（分析内容）@总结：（总结内容）。总结部分仅给出上升下降趋势，幅度，时间，如何波动等销量变化信息，不要提及其他城市，尽量简洁清晰。"
    
    response = conversation.run(input=prompt)
    logging.info(f"{cityidx}, {item} \n{prompt} \n\n{response} \n")
    no_response_dict[(cityidx, item)] = response

    t2 = time.time()
    print(i, cityidx, idx_to_city[cityidx], t2-t1)
    if response == "":
        print("!"*50)

with open('./data/no_response_dict.pkl', 'wb') as pickle_file:
    pickle.dump(no_response_dict, pickle_file)

In [ ]:
with open('./data/no_response_dict.pkl', 'rb') as pickle_file:
    no_response_dict = pickle.load(pickle_file)

In [ ]:
all_response_dict = {}
for key in response_dict:
    all_response_dict[key] = no_response_dict[key] if key in no_response_dict else response_dict[key]
with open('./data/all_response_dict.pkl', 'wb') as pickle_file:
    pickle.dump(all_response_dict, pickle_file)

In [ ]:
with open('./data/all_response_dict.pkl', 'rb') as pickle_file:
    all_response_dict = pickle.load(pickle_file)

In [ ]:
# embed_dict = {}
embed_tensor = torch.zeros((337, 26, 768))

keys = list(all_response_dict.keys())
texts = list(all_response_dict.values())
texts = [extract_summary(text) for text in texts]

batch = 2048
for i in range(0, len(texts), batch):
    batch_keys = keys[i:min(i + batch, len(texts))]
    batch_texts = texts[i:min(i + batch, len(texts))]
    sentence_embeddings = text_embed(batch_texts).cpu()
    # 更新嵌入字典
    for key, embed in zip(batch_keys, sentence_embeddings):
        # embed_dict[key] = embed
        embed_tensor[key[0]][key[1]] = embed
    print(i)

In [ ]:
# with open('./data/embed_dict.pkl', 'wb') as pickle_file:
#     pickle.dump(embed_dict, pickle_file)
# with open('./data/embed_dict.pkl', 'rb') as pickle_file:
#     embed_dict = pickle.load(pickle_file)
torch.save(embed_tensor, './data/embed_tensor.pt')

In [ ]:
embed_tensor = torch.load('./data/embed_tensor.pt')

## case

In [ ]:
item = 1
city = 254

cityidxlist = similarcity_dict[city]
print(idx_to_city[city], idx_to_item[item], [idx_to_city[idx] for idx in cityidxlist])

In [ ]:
tt = np.array([[round(num, 4) for num in num_dict[(idx, item)].numpy().tolist()] for idx in cityidxlist])
tt.shape

In [ ]:
data = np.array([[round(num, 4) for num in num_dict[(idx, item)].numpy().tolist()] for idx in [city]])
data.shape

In [ ]:
def min_max_normalize(sub_sequence):
    min_val = np.min(sub_sequence[:14])
    max_val = np.max(sub_sequence[:14])
    # 避免除以零的情况
    if max_val - min_val == 0:
        return sub_sequence  # 如果所有值相同，返回原序列
    return (sub_sequence - min_val) / (max_val - min_val)

In [ ]:
tt = np.array([min_max_normalize(ttt) for ttt in tt])
data = min_max_normalize(data)
tt.shape, data.shape

In [ ]:
def dtw_distance(sequence1, sequence2):
    n = len(sequence1)
    m = len(sequence2)
    # 创建一个 (n+1) x (m+1) 的距离矩阵，初始化为无穷大
    dtw_matrix = np.full((n + 1, m + 1), np.inf)
    dtw_matrix[0, 0] = 0

    # 填充 DTW 矩阵
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = abs(sequence1[i - 1] - sequence2[j - 1])
            # 选择最小的累积距离路径
            dtw_matrix[i, j] = cost + min(
                dtw_matrix[i - 1, j],    # 插入
                dtw_matrix[i, j - 1],    # 删除
                dtw_matrix[i - 1, j - 1] # 匹配
            )

    # 返回 DTW 距离
    return dtw_matrix[n, m]

# # 示例序列
# sequence_a = np.random.rand(28)
# sequence_b = np.random.rand(28)

# # 计算 DTW 距离
# distance = dtw_distance(sequence_a, sequence_b)
# print("DTW distance:", distance)

In [ ]:
# 计算每个序列与目标序列的 DTW 距离
distances = []
for i, sequence in enumerate(tt):
    distance = dtw_distance(sequence, data[0])
    distances.append((distance, i))

# 按距离排序，找出最小的 5 个距离对应的序列索引
distances.sort(key=lambda x: x[0])

In [ ]:
top5idx = [idx for _, idx in distances[:5]]
bottom5idx = [idx for _, idx in distances[-5:]]
top5idx, bottom5idx

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 创建图形和子图
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(6, 5))
axes = axes.flatten()  # 将子图对象展平成一维数组，方便迭代


ax = axes[0]
# 绘制前14天
ax.plot(range(15), data[0, :15], color='blue')
# 绘制后14天
ax.plot(range(14, 28), data[0, 14:], color='red')
# 添加标题和图例
ax.set_title(f'{0}')

# 可视化每个时间序列
for i, idx in enumerate(top5idx):
    ax = axes[i+1]
    time_series = tt[idx]
    
    # 绘制前14天
    ax.plot(range(15), time_series[:15], color='blue')
    
    # 绘制后14天
    ax.plot(range(14, 28), time_series[14:], color='red')
    
    # 添加标题和图例
    ax.set_title(f'{i+1}')
    #ax.legend()

# 调整布局
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 创建图形和子图
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(6, 5))
axes = axes.flatten()  # 将子图对象展平成一维数组，方便迭代


ax = axes[0]
# 绘制前14天
ax.plot(range(15), data[0, :15], color='blue')
# 绘制后14天
ax.plot(range(14, 28), data[0, 14:], color='red')
# 添加标题和图例
ax.set_title(f'{0}')

# 可视化每个时间序列
for i, idx in enumerate(bottom5idx):
    ax = axes[i+1]
    time_series = tt[idx]
    
    # 绘制前14天
    ax.plot(range(15), time_series[:15], color='blue')
    
    # 绘制后14天
    ax.plot(range(14, 28), time_series[14:], color='red')
    
    # 添加标题和图例
    ax.set_title(f'{i+1}')
    #ax.legend()

# 调整布局
plt.tight_layout()
plt.show()

In [ ]:
tt[top5idx].shape, tt[bottom5idx].shape, data.shape

In [ ]:
top5city = [idx_to_city[idx] for idx in [cityidxlist[id] for id in top5idx]]
bottom5city = [idx_to_city[idx] for idx in [cityidxlist[id] for id in bottom5idx]]
top5city, bottom5city

In [ ]:
# 保存多个数组到一个 .npz 文件
np.savez(f'./data/case_city{city}_item{item}.npz', top5city=tt[top5idx], othercity=tt[bottom5idx], targetcity=data[0])

In [ ]:
# 加载 .npz 文件
with np.load('./data/case_city254_item1.npz') as data:
    top5city = data['top5city'] # 5*28
    othercity = data['othercity'] # 5*28
    targetcity = data['targetcity'] # 28
print(top5city.shape, othercity.shape, targetcity.shape)